In [1]:
!pip install yfinance
!pip install yahoofinancials

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 62 kB 1.2 MB/s 
  Attempting uninstall: requests
    Found existing installation: requests 2.23.0
    Uninstalling requests-2.23.0:
      Successfully uninstalled requests-2.23.0
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for yahoofinancials: filename=yahoofinancials-1.6-py3-none-any.whl size=15191 sha256=83e880a16ead2c1cca671a93393b0f9d4bdf65ecaad519ab128b54b2198568a4
  Stored in directory: /root/.cache/pip/wheels/4b/63/46/e7110bfee88685fe69e338d1b14d1748921862aa57b6705b60
Successfully built yahoofinancials


In [5]:
import numpy as np
import time as tm
import datetime 
import tensorflow as tf
import pandas as pd
# Data preparation
from sklearn.preprocessing import MinMaxScaler
from collections import deque

# AI
from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout

# Graphics library
import matplotlib.pyplot as plt
import yfinance as yf
from yahoofinancials import YahooFinancials
import time
import datetime

In [3]:
#time to pull the data
start_date =datetime.datetime.today().strftime('%Y-%m-%d')
end_date = (datetime.datetime.today() - datetime.timedelta(days=1100)).strftime('%Y-%m-%d')

In [6]:
# calling API 
data=YahooFinancials('goog').get_historical_price_data(end_date,start_date,"daily")
#into the dataframe
goog_df = pd.DataFrame(data['GOOG']['prices'])
goog_df =goog_df.drop(['high','low','open','volume','date','adjclose'], axis=1)
goog_df.rename(columns={'formatted_date':'date'},inplace=True)
goog_df['date']=pd.to_datetime(goog_df['date'])

In [7]:
# SETTINGS

# Window size or the sequence length, 7 (1 week)
N_STEPS = 7

# Lookup steps, 1 is the next day, 3 = after tomorrow
LOOKUP_STEPS = [1, 2, 3]

In [8]:
# Scale data for ML engine
scaler = MinMaxScaler()
goog_df['scaled_close'] = scaler.fit_transform(np.expand_dims(goog_df['close'].values, axis=1))

In [9]:
def PrepareData(days):
    df = goog_df.copy()
    df['future'] = df['scaled_close'].shift(-days)
    last_sequence = np.array(df[['scaled_close']].tail(days))
    df.dropna(inplace=True)
    sequence_data = []
    sequences = deque(maxlen=N_STEPS)
    
    for entry, target in zip(df[['scaled_close'] + ['date']].values, df['future'].values):
        sequences.append(entry)
        if len(sequences) == N_STEPS:
            sequence_data.append([np.array(sequences), target])
            
    last_sequence = list([s[:len(['scaled_close'])] for s in sequences]) + list(last_sequence)
    last_sequence = np.array(last_sequence).astype(np.float32)

    # construct the X's and Y's
    X, Y = [], []
    for seq, target in sequence_data:
        X.append(seq)
        Y.append(target)

   # convert to numpy arrays
    X = np.array(X)
    Y = np.array(Y)

    return df, last_sequence, X, Y

In [10]:
PrepareData(3) # 3 days


(         close       date  scaled_close    future
 0    65.443001 2019-11-07      0.128854  0.123715
 1    65.568497 2019-11-08      0.130136  0.123307
 2    64.959503 2019-11-11      0.123914  0.130182
 3    64.940002 2019-11-12      0.123715  0.142141
 4    64.900002 2019-11-13      0.123307  0.134903
 ..         ...        ...           ...       ...
 751  90.500000 2022-11-01      0.384857  0.346033
 752  87.070000 2022-11-02      0.349813  0.365956
 753  83.489998 2022-11-03      0.313237  0.368612
 754  86.699997 2022-11-04      0.346033  0.353185
 755  88.650002 2022-11-07      0.365956  0.422352
 
 [756 rows x 4 columns], array([[0.4469748 ],
        [0.42735857],
        [0.38485664],
        [0.349813  ],
        [0.31323686],
        [0.34603277],
        [0.3659556 ],
        [0.36861196],
        [0.35318458],
        [0.42235228]], dtype=float32), array([[[0.12885428517199038, Timestamp('2019-11-07 00:00:00')],
         [0.13013645187704637, Timestamp('2019-11-08 00:00:0

In [11]:
def GetTrainedModel(x_train, y_train):
  model = Sequential()
  model.add(LSTM(60, return_sequences=True, input_shape=(N_STEPS, len(['scaled_close']))))
  model.add(Dropout(0.3))
  model.add(LSTM(120, return_sequences=False))
  model.add(Dropout(0.3))
  model.add(Dense(20))
  model.add(Dense(1))

  BATCH_SIZE = 8
  EPOCHS = 80

  model.compile(loss='mean_squared_error', optimizer='adam')

  model.fit(x_train, y_train,
            batch_size=BATCH_SIZE,
            epochs=EPOCHS,
            verbose=1)

  model.summary()

  return model

In [12]:
# GET PREDICTIONS
predictions = []

for step in LOOKUP_STEPS:
  df, last_sequence, x_train, y_train = PrepareData(step)
  x_train = x_train[:, :, :len(['scaled_close'])].astype(np.float32)

  model = GetTrainedModel(x_train, y_train)

  last_sequence = last_sequence[-N_STEPS:]
  last_sequence = np.expand_dims(last_sequence, axis=0)
  prediction = model.predict(last_sequence)
  predicted_price = scaler.inverse_transform(prediction)[0][0]

  predictions.append(round(float(predicted_price), 2))

Epoch 1/80
94/94 [==============================] - 4s 10ms/step - loss: 0.0210
Epoch 2/80
94/94 [==============================] - 1s 10ms/step - loss: 0.0051
Epoch 3/80
94/94 [==============================] - 1s 10ms/step - loss: 0.0043
Epoch 4/80
94/94 [==============================] - 1s 10ms/step - loss: 0.0032
Epoch 5/80
94/94 [==============================] - 1s 10ms/step - loss: 0.0034
Epoch 6/80
94/94 [==============================] - 1s 10ms/step - loss: 0.0030
Epoch 7/80
94/94 [==============================] - 1s 10ms/step - loss: 0.0030
Epoch 8/80
94/94 [==============================] - 1s 10ms/step - loss: 0.0028
Epoch 9/80
94/94 [==============================] - 1s 10ms/step - loss: 0.0026
Epoch 10/80
94/94 [==============================] - 1s 10ms/step - loss: 0.0025
Epoch 11/80
94/94 [==============================] - 1s 10ms/step - loss: 0.0022
Epoch 12/80
94/94 [==============================] - 1s 10ms/step - loss: 0.0023
Epoch 13/80
94/94 [==================

In [13]:
if bool(predictions) == True and len(predictions) > 0:
  predictions_list = [str(d)+'$' for d in predictions]
  predictions_str = ', '.join(predictions_list)
  message = f' prediction for upcoming 3 days ({predictions_str})'
  
  print(message)

 prediction for upcoming 3 days (90.7$, 92.04$, 90.76$)


In [15]:
from google.colab import files
today=start_date =datetime.datetime.today()
day1=(today+datetime.timedelta(days=1)).strftime('%Y-%m-%d')
day2=(today+datetime.timedelta(days=2)).strftime('%Y-%m-%d')
day3=(today+datetime.timedelta(days=3)).strftime('%Y-%m-%d')
days=[day1,day2,day3]
goog_lstm=pd.DataFrame()
goog_lstm['future days']=days
goog_lstm['price']=predictions
goog_lstm['ticker']='GOOG'

goog_lstm.to_csv('goog_lstm.csv') 
files.download('goog_lstm.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>